In [47]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os
import time
import numpy as np
import re
from tqdm import tqdm


In [46]:
sleep_min = 1
sleep_max = 3
client_id = "72c9f2b9d3344c91af50a24f4144228f"
client_secret = "249943a9bbb74b9a8bf04bb7fd93790d"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # sp is the instance of the spotipy api

In [7]:
data = pd.read_csv('/Users/ytkd/Desktop/LY_Artist_lyrics_genre_data_from_big5_mft_users_likes_final.csv')
data = data[data['lang_detect_spacy']=='en']
sample_dataset = data[:200]

</h2> Data Cleaning


In [15]:
artists = list(sample_dataset['Artist'])
titles = list(sample_dataset['title'])

In [84]:
def pre_process_text(text):
    text = text.lower()
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    return text


In [113]:
'''
check how many artists and titles have special characters
'''

'\ncheck how many artists and titles have special characters\n'

In [85]:
artists_clean = []
titles_clean = []

for i in tqdm(range(len(artists))):
    artists_clean.append(pre_process_text(artists[i]))
    titles_clean.append(pre_process_text(titles[i]))

100%|██████████| 200/200 [00:00<00:00, 110550.98it/s]


In [86]:
request_count = 0
start_time = time.time()
spotify_artists = []
check_dat = []
for i in tqdm(range(len(artists))):
    query = sp.search(f'artist:{artists_clean[i]} track:{titles_clean[i]}')
    if query['tracks']['items']:
        spotify_artists.append(query['tracks']['items'][0]['artists'][0]['name'])
        check_dat.append(query['tracks']['items'][0]['artists'][0]['name'])
    else:
        spotify_artists.append(artists[i])
        
    request_count+=1
    if request_count % 5 == 0:
        # print(str(request_count) + " requests sent")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        # print('Elapsed Time: {} seconds'.format(time.time() - start_time))
print('Completed...')

100%|██████████| 200/200 [01:53<00:00,  1.77it/s]

Completed...


In [88]:
print(f'Out of the 200 sample data records, data for {len(check_dat)} records is available')

Out of the 200 sample data records, data for 185 records is available


In [89]:
df1 = pd.DataFrame(spotify_artists)
df2 = pd.DataFrame(artists)
df3 = pd.concat([df2,df1], axis=1, join='outer')
df3.to_csv('check.csv')

Checking if the artisrts that we extracted from spotify are same as in our dataset

In [90]:
count = 0
for i in tqdm(range(len(artists))):
    if pre_process_text(artists[i]) in pre_process_text(spotify_artists[i]):
        count+=1
count

100%|██████████| 200/200 [00:00<00:00, 28263.50it/s]


199

Extracting Data

In [91]:
'''
In track_preview_available function I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available.

In is_same_artist function I check whether the artist the we found in our query is the same one from our dataset.
'''
def is_same_artist(query, data, index):
    if pre_process_text(data[index]) in pre_process_text(query['tracks']['items'][0]['artists'][0]['name']):
        return True

def track_preview_available(query):
    if query['tracks']['items']:
        if query['tracks']['items'][0]['preview_url']:
            return True
        return True
        
def artist_available(query):
    if query['tracks']['items']:
        return True

In [141]:
query = sp.search('coldplay fix you', type='track')
query['tracks']['items'][0]['artists'][0]['id']

'4gzpq5DPGxSnKTe4SA8HAU'

In [142]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()

preview_url = []
track_id = []
artist_id = []
track_popularity = []
artists_df = []
titles_df = []

for index in range(len(spotify_artists)):
    artist_name = spotify_artists[index]
    song_title = titles_clean[index]
    search = f'artist:{artist_name} track:{song_title}'
    query = sp.search(search, type='track')


    # in the below line of code I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available, and
    # even further I check a third condition whether the artist the we found in our query is the same from our dataset.


    # if query['tracks']['items'] and query['tracks']['items'][0]['preview_url'] and query['tracks']['items'][0]['artists'][0]['name'] in alpha_artists_titles[index]:
    if track_preview_available(query) and is_same_artist(query,artists,index):
        preview_url.append(query['tracks']['items'][0]['preview_url'])
        track_id.append(query['tracks']['items'][0]['id'])
        artist_id.append(query['tracks']['items'][0]['artists'][0]['id'])
        track_popularity.append(query['tracks']['items'][0]['popularity'])
        artists_df.append(spotify_artists[index])
        titles_df.append(titles_clean[index])
        
        # response = requests.get(query['tracks']['items'][0]['preview_url'], verify=False) # using track uris
        # audio_fn = os.path.join(audio_path,*track_id[:2],track_id+'.mp3') 
        # open(f"downloaded_songs/{artists_10[i]}/{top_10_track_names_clean[song]}.mp3", 'wb').write(response.content)   
    else:
        preview_url.append(None)
        track_id.append(None)
        artist_id.append(None)
        track_popularity.append(None)
        artists_df.append(spotify_artists[index])
        titles_df.append(titles_clean[index])


    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 artists uris fetched
Loop #: 5
Elapsed Time: 3.1694400310516357 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 6.292775869369507 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 9.357357025146484 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 11.815310955047607 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 13.77209997177124 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 16.458618879318237 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 18.793312072753906 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 21.351402044296265 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 24.717917919158936 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 26.63834500312805 seconds
55 artists uris fetched
Loop #: 55
Elapsed Time: 29.00997495651245 seconds
60 artists uris fetched
Loop #: 60
Elapsed Time: 32.62279200553894 seconds
65 artists uris fetched
Loop #: 65
Elapsed Time: 35.80810189247131 seconds
70 artists uris fetch

In [143]:
track_data = pd.DataFrame(artists_df, columns=['artists'])
track_data['tracks'] = titles_df
track_data['artist_id'] = artist_id
track_data['track_id'] = track_id
track_data['track_popularity'] = track_popularity
track_data['track_url'] = preview_url

In [144]:
track_data

,artists,tracks,artist_id,track_id,track_popularity,track_url
0,*NSYNC,bye bye bye,6Ff53KvcvAj5U7Z1vojB5o,62bOmKYxYg7dhrC6gH9vFn,70.0,https://p.scdn.co/mp3-preview/612fd571c0216fa3...
1,*NSYNC,it s gonna be me,6Ff53KvcvAj5U7Z1vojB5o,2AW37v0bDyuOzGP3XnmFuA,67.0,https://p.scdn.co/mp3-preview/f3bdcf480e1b5fcc...
2,*NSYNC,tearin up my heart,6Ff53KvcvAj5U7Z1vojB5o,73q3FpQVXWk5eSUnyo83E4,64.0,https://p.scdn.co/mp3-preview/9e9edd302e6a06be...
3,*NSYNC,gone,6Ff53KvcvAj5U7Z1vojB5o,4CCUjYJPbSXLL23BFeBVbI,52.0,https://p.scdn.co/mp3-preview/eea62070ce877996...
4,*NSYNC,"merry christmas, happy holidays",6Ff53KvcvAj5U7Z1vojB5o,4v9WbaxW8HdjqfUiWYWsII,39.0,https://p.scdn.co/mp3-preview/158f2eafaa4f03f5...
...,...,...,...,...,...,...
195,65daysofstatic,heat death infinity splitter,6DVVsQAnpHdJjb1nYuOQ6g,3PibgbUWMGaYZObGpvGhmJ,26.0,https://p.scdn.co/mp3-preview/2799b9f6b7e4fe22...
196,Three 6 Mafia,sippin on some syrup,26s8LSolLfCIY88ysQbIuT,2EduiM2UNwjCC7PsLxnOCH,58.0,https://p.scdn.co/mp3-preview/2430a27f21f8d77f...
197,Three 6 Mafia,stay fly,26s8LSolLfCIY88ysQbIuT,5MYFw4T2gy52pOGBN4EYHS,68.0,https://p.scdn.co/mp3-preview/56db43014efeb814...
198,Three 6 Mafia,late nite tip,26s8LSolLfCIY88ysQbIuT,75RK78POyFmg3u6O1cpBdr,51.0,https://p.scdn.co/mp3-preview/3e7e1d015ddb7feb...


In [145]:
# Downloading these songs according to track id
track_data.set_index('track_id',inplace=True)
track_data

,artists,tracks,artist_id,track_popularity,track_url
track_id,,,,,
62bOmKYxYg7dhrC6gH9vFn,*NSYNC,bye bye bye,6Ff53KvcvAj5U7Z1vojB5o,70.0,https://p.scdn.co/mp3-preview/612fd571c0216fa3...
2AW37v0bDyuOzGP3XnmFuA,*NSYNC,it s gonna be me,6Ff53KvcvAj5U7Z1vojB5o,67.0,https://p.scdn.co/mp3-preview/f3bdcf480e1b5fcc...
73q3FpQVXWk5eSUnyo83E4,*NSYNC,tearin up my heart,6Ff53KvcvAj5U7Z1vojB5o,64.0,https://p.scdn.co/mp3-preview/9e9edd302e6a06be...
4CCUjYJPbSXLL23BFeBVbI,*NSYNC,gone,6Ff53KvcvAj5U7Z1vojB5o,52.0,https://p.scdn.co/mp3-preview/eea62070ce877996...
4v9WbaxW8HdjqfUiWYWsII,*NSYNC,"merry christmas, happy holidays",6Ff53KvcvAj5U7Z1vojB5o,39.0,https://p.scdn.co/mp3-preview/158f2eafaa4f03f5...
...,...,...,...,...,...
3PibgbUWMGaYZObGpvGhmJ,65daysofstatic,heat death infinity splitter,6DVVsQAnpHdJjb1nYuOQ6g,26.0,https://p.scdn.co/mp3-preview/2799b9f6b7e4fe22...
2EduiM2UNwjCC7PsLxnOCH,Three 6 Mafia,sippin on some syrup,26s8LSolLfCIY88ysQbIuT,58.0,https://p.scdn.co/mp3-preview/2430a27f21f8d77f...
5MYFw4T2gy52pOGBN4EYHS,Three 6 Mafia,stay fly,26s8LSolLfCIY88ysQbIuT,68.0,https://p.scdn.co/mp3-preview/56db43014efeb814...
